In [1]:
import datetime
from datetime import timedelta
import numpy as np
import os
import pandas as pd
import time
import yahoo_fin.stock_info as si

class Parser:
    def __init__(self, stock_file_name_constructor, requests_filename,\
                 stock_df_header, first_request_history_days = 30):
        self.last_request_dict = {}
        
        self.stock_file_name_constructor = stock_file_name_constructor 
        self.requests_filename = requests_filename 
        self.stock_df_header = stock_df_header
        self.update_threshold = datetime.timedelta(days = 1)
        self.first_request_history_days = first_request_history_days
        
    def update_history(self):
        current_date = datetime.datetime.now()
        if (os.path.isfile(self.requests_filename)):
            requests = open(self.requests_filename, "r")
            requests_heading = requests.readline()
            for raw_row in requests:
                row = raw_row.split(",")
                stock_name = row[0]
                stock_file_name = self.stock_file_name_constructor(stock_name)
                try:
                    if stock_name in self.last_request_dict:
                        start_date = self.last_request_dict[stock_name]
                    else:
                        start_date = current_date - datetime.timedelta(days=self.first_request_history_days)
                        stock_file = open(stock_file_name, "w")
                        stock_file.write(self.stock_df_header)
                        stock_file.close()
                    if (current_date - start_date > self.update_threshold):
                        historical_prices = si.get_data(stock_name, start_date, current_date)
                        historical_prices.to_csv(path_or_buf = stock_file_name, header = False, mode = 'a')
                        self.last_request_dict[stock_name] = current_date
                except Exception as e:
                    print(f'Error {e} has occured')
            requests.close()
            os.remove(self.requests_filename)

def history_stock_df_name(stock_name):
    return "history_" + stock_name + ".csv"

requests_filename = "current_requests.csv"
stock_df_header = "Datetime,Open,High,Low,Close,Adj Close,Volume,Ticker\n"

default_parser = Parser(stock_file_name_constructor = history_stock_df_name,\
                        requests_filename = requests_filename,\
                        stock_df_header = stock_df_header)


In [2]:
default_parser.update_history(25)
default_parser.update_history(20)
default_parser.update_history(15)
default_parser.update_history(10)
default_parser.update_history(5)
default_parser.update_history(1)

In [8]:
import datetime
import yahoo_fin.stock_info as si

end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=14)
historical_prices = si.get_data("AAPL", start_date, end_date)

historical_prices

,open,high,low,close,adjclose,volume,ticker
2023-12-11,193.110001,193.490005,191.419998,193.179993,193.179993,60943700,AAPL
2023-12-12,193.080002,194.720001,191.720001,194.710007,194.710007,52696900,AAPL
2023-12-13,195.089996,198.000000,194.850006,197.960007,197.960007,70404200,AAPL
2023-12-14,198.020004,199.619995,196.160004,198.110001,198.110001,66831600,AAPL
2023-12-15,197.529999,198.399994,197.000000,197.570007,197.570007,128256700,AAPL
2023-12-18,196.089996,196.630005,194.389999,195.889999,195.889999,55751900,AAPL
2023-12-19,196.160004,196.949997,195.889999,196.940002,196.940002,40714100,AAPL
2023-12-20,196.899994,197.679993,194.830002,194.830002,194.830002,52242800,AAPL
2023-12-21,196.100006,197.080002,193.500000,194.679993,194.679993,46482500,AAPL
2023-12-22,195.179993,195.410004,192.970001,193.600006,193.600006,37122800,AAPL
